In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from torchHHT import hht, visualization
from scipy.signal import chirp
import IPython
fs = 1000
duration = 2.0
t = torch.arange(fs*duration) / fs
x = torch.from_numpy(chirp(t, 5, 0.8, 10, method = "quadratic", phi=100)) * torch.exp(-4*(t-1)**2) + \
    torch.from_numpy(chirp(t, 40, 1.2, 50, method = "linear")) * torch.exp(-4*(t-1)**2)

plt.plot(t, x) 
plt.title("$x(t)$")
plt.xlabel("time")
plt.show()
imfs, imfs_env, imfs_freq = hht.hilbert_huang(x, fs, num_imf=3)
visualization.plot_IMFs(x, imfs, fs, save_fig="img/emd.png")
spectrum, t, f = hht.hilbert_spectrum(imfs_env, imfs_freq, fs, freq_lim = (0, 60), time_scale=1, freq_res = 1)
visualization.plot_HilbertSpectrum(spectrum, t, f, 
                                    save_spectrum="img/Hilbert_spectrum.png", 
                                    save_marginal="img/Hilbert_marginal.png")
from scipy.signal import stft
from torch import fft
plt.figure(figsize=(20, 4))

f, t, Zxx = stft(x, fs, nperseg=1024, noverlap=1023, nfft=1024)
f_lim = int(60/f[1])
ax = plt.subplot(1, 3, 1)
plt.colorbar(ax.pcolormesh(t, f[:f_lim], 20 * np.log10(np.abs(Zxx))[:f_lim, :], shading='auto', cmap = plt.cm.jet),            
            label="energy(dB)")
ax.set_xlabel("time")
ax.set_ylabel("frequency")
ax.set_title("(a) Spectrogram (long window)")

f, t, Zxx = stft(x, fs, nperseg=128, noverlap=127, nfft = 1024)
f_lim = int(60/f[1])
ax = plt.subplot(1, 3, 2)
plt.colorbar(ax.pcolormesh(t, f[:f_lim], 20 * np.log10(np.abs(Zxx))[:f_lim, :], shading='auto', cmap = plt.cm.jet), 
            label="energy(dB)")
ax.set_xlabel("time")
ax.set_ylabel("frequency")
ax.set_title("(b) Spectrogram (short window)")

X = fft.fft(x)
ax = plt.subplot(1, 3, 3)
f_lim = int(100/fs*x.shape[0])
ax.plot(np.arange(f_lim)/x.shape[0]*fs, 20 * np.log10(np.abs(X))[:f_lim])
ax.set_xlim(0, 60)
ax.set_xlabel("frequency")
ax.set_ylabel("energy (dB)")
ax.set_title("(c) marginal spectrum.")
plt.savefig("img/STFT_spectrum.png", dpi = 600)
plt.show()

ModuleNotFoundError: No module named 'torchHHT'